In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time
import glob
import warnings
warnings.filterwarnings('ignore')


#data
path = "./data/data_daily"
all_files = glob.glob(path + "/*.csv")

li = []


for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)


frame = pd.concat(li, axis=0, ignore_index=True)
result_df = frame.sort_values(by=['code', 'date'], ascending=True)
result_df['vwap'] = result_df['money'] / result_df['volume']   #基础行情生成
df = result_df.copy()

In [2]:
import pandas as pd
df = pd.read_csv('./data/new_daily7/2021-06-30.csv')
df

,code,close_mean,vwap_std,pro_vol_sum,gap_vol_sum,date,open,close,low,high,volume,money,turnover_ratio
0,000001.XSHE,2385.148488,6.157922,2.462618e+05,2.209435e+05,2021-06-30,2721.64,2753.29,2720.42,2791.02,331448.0,9.132852e+08,0.2079
1,000002.XSHE,2917.717298,3.816815,3.537453e+05,2.894711e+05,2021-06-30,3407.11,3405.68,3361.34,3421.41,348782.0,1.183027e+09,0.5134
2,000004.XSHE,109.165169,0.342200,4.937935e+05,3.816267e+05,2021-06-30,137.30,138.20,136.34,140.21,625490.0,8.637627e+07,4.0455
3,000005.XSHE,18.462583,0.045766,2.739191e+05,2.498120e+05,2021-06-30,19.32,19.22,19.01,19.53,577443.0,1.109170e+07,0.5580
4,000006.XSHE,346.643965,0.257050,2.682418e+04,3.390514e+04,2021-06-30,248.56,246.59,245.61,248.56,92604.0,2.283820e+07,0.3376
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4333,688777.XSHG,93.343437,0.936331,1.255925e+06,1.265914e+06,2021-06-30,90.21,94.42,89.32,95.08,1894646.0,1.766767e+08,4.6288
4334,688788.XSHG,146.131683,2.050037,8.600470e+05,7.115330e+05,2021-06-30,138.07,146.22,137.11,149.35,1349500.0,1.959448e+08,7.7977
4335,688819.XSHG,45.089231,0.269061,2.334119e+06,2.259252e+06,2021-06-30,44.04,45.15,43.95,45.64,4656323.0,2.096566e+08,4.6754
4336,688981.XSHG,61.712500,0.696921,4.733407e+07,4.230815e+07,2021-06-30,59.69,61.82,59.40,62.50,77795053.0,4.787665e+09,7.1003


In [ ]:
#日频截面基础字段生成（仅作示例，更多字段大家可以自行生成）

#价格振幅
df['Amplitude_oc'] = np.abs(df['close'] - df['open'])
df['Amplitude_hl'] = np.abs(df['high'] - df['low'])

#价格涨跌幅
df['pct_oc'] = (df['close'] - df['open']) / df['open']
df['pct_hl'] = (df['high'] - df['low']) / df['low']

#价格均值
df['mean_oc'] = (df['close'] + df['open']) / 2
df['mean_hl'] = (df['high'] + df['low']) / 2

#其他价格字段
df['vwap_close'] = df['vwap'] / df['close']

#日k线最短路径
df['Spath'] = 2*(df['high'] - df['low']) - np.abs(df['close'] - df['open'])

#上下影线
df['Ushadow'] = df['high'] - (df[['close', 'open']].max(axis=1))
df['Lshadow'] = (df[['close', 'open']].min(axis=1)) - df['low']

#标准化上下影线
df['SUshadow'] = (df['high'] - (df[['close', 'open']].max(axis=1))) / df['high']
df['SLshadow'] = ((df[['close', 'open']].min(axis=1)) - df['low']) / df['low']

#截面排序      注意排序的字段在量级上要一致
df['r_turnover_ratio'] = df.groupby('date')['turnover_ratio'].rank(pct=True).values

df['r_vc'] = df.groupby('date')['vwap_close'].rank(pct=True).values

df['r_pct_oc'] = df.groupby('date')['pct_oc'].rank(pct=True).values

df['r_pct_hl'] = df.groupby('date')['pct_hl'].rank(pct=True).values
